In [ ]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [53]:
import turicreate
import pandas as pd

In [54]:
products = turicreate.SFrame('~/Courses/u-wash-machine-learning/machine-learning-case-study/data/amazon_baby.sframe')

In [155]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [55]:
products = products.to_dataframe()

In [109]:
products.head()

,name,review,rating,review_clean,awesome,great,fantastic,amazing,love,horrible,bad,terrible,awful,wow,hate,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5.0,it came early and was not disappointed. i love...,0,0,0,0,1,0,0,0,0,0,0,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5.0,Very soft and comfortable and warmer than it l...,0,0,0,0,0,0,0,0,0,0,0,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5.0,This is a product well worth the purchase. I ...,0,0,0,0,2,0,0,0,0,0,0,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5.0,All of my kids have cried non-stop when I trie...,0,1,0,0,0,0,0,0,0,0,0,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5.0,"When the Binky Fairy came to our house, we did...",0,1,0,0,0,0,0,0,0,0,0,1


In [111]:
def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator) 

In [112]:
products['review_clean'] = products['review'].astype(str).apply(remove_punctuation) #astype(str) makes sure all reviews are strings

In [113]:
products['review_clean']

1         it came early and was not disappointed i love ...
2         Very soft and comfortable and warmer than it l...
3         This is a product well worth the purchase  I h...
4         All of my kids have cried nonstop when I tried...
5         When the Binky Fairy came to our house we didn...
                                ...                        
183526    Such a great idea very handy to have and look ...
183527    This product rocks  It is a great blend of fun...
183528    This item looks great and cool for my kidsI kn...
183529    I am extremely happy with this product I have ...
183530    I love this product very mush  I have bought m...
Name: review_clean, Length: 166752, dtype: object

In [114]:
for word in selected_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

In [115]:
products[selected_words]

,awesome,great,fantastic,amazing,love,horrible,bad,terrible,awful,wow,hate
1,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
183526,0,2,0,0,0,0,0,0,0,0,0
183527,0,1,0,0,0,0,0,0,0,0,0
183528,0,2,0,0,0,0,0,0,0,0,0
183529,0,0,0,0,0,0,0,0,0,0,0


Using the .sum() method on each of the new columns you created, answer the following questions:  Out of the selected_words, which one is most used in the dataset?  Which one is least used? Save these results to answer the quiz at the end.

In [116]:
# Most = great
# Least = wow 
for w in selected_words:
    print(w, products[w].sum())

awesome 3078
great 46018
fantastic 1418
amazing 2148
love 32999
horrible 940
bad 3911
terrible 970
awful 565
wow 97
hate 970


## Create a new sentiment analysis model using only the selected_words as features
In the Jupyter Notebook above, we used word counts for all words as features for our sentiment classifier.  Now, you are just going to use the selected_words:

In [117]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [118]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [119]:
products['sentiment'].sum()

113766

### Train Sentiment Classifier

In [120]:
import numpy as np
from sklearn.model_selection import train_test_split

#Split Data
train_data , test_data = train_test_split(products, test_size=0.2)

In [121]:
from sklearn.linear_model import LogisticRegression

In [122]:
logisticRegr_mdl = LogisticRegression()

In [123]:
selected_words_model = logisticRegr_mdl.fit(train_data[selected_words], train_data['sentiment'])

In [124]:
selected_words_model.coef_

array([[ 1.08562011,  0.75935287,  0.77985457,  0.92946395,  1.30971533,
        -2.17753433, -0.94453235, -2.03114099, -1.94442613, -0.56208869,
        -1.33141153]])

In [125]:
weight = selected_words_model.coef_

In [94]:
flat_list = [item for sublist in weight.reshape(-1,1).tolist() for item in sublist]

In [126]:
print(selected_words)
flat_list

['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']


[0.880178973826363,
 0.7567906011915277,
 0.7027407398959553,
 0.9677355989010565,
 1.361358635122838,
 -1.9147221373633176,
 -0.938747037543111,
 -1.9034994280341926,
 -1.5068577812275281,
 -0.32869041567227136,
 -1.4065833182043057]

In [127]:
feature_weight = pd.DataFrame(
    {'feature': selected_words,
     'weight': flat_list
    })
feature_weight

,feature,weight
0,awesome,0.880179
1,great,0.756791
2,fantastic,0.702741
3,amazing,0.967736
4,love,1.361359
5,horrible,-1.914722
6,bad,-0.938747
7,terrible,-1.903499
8,awful,-1.506858
9,wow,-0.328690


What is the accuracy of the selected_words_model on the test_data?  What was the accuracy of the sentiment_model that we learned using all the word counts in the Jupyter Notebook above from the lectures?  What is the accuracy majority class classifier on this task?  How do you compare the different learned models with the baseline approach where we are just predicting the majority class?    Save these results to answer the quiz at the end.

In [128]:
from sklearn.metrics import accuracy_score

y_pred_selected_words = selected_words_model.predict(test_data[selected_words])
y_true_selected_words = test_data['sentiment']
print('answer: the accuracy of selected words model is',accuracy_score(y_true_selected_words, y_pred_selected_words))

answer: the accuracy of selected words model is 0.8468711582861084


We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

In [129]:
from sklearn.feature_extraction.text import CountVectorizer

#single letter words
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b') 
#learn vocabulary from the training data and assign columns to words, convert to sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])

#convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [156]:
print(train_matrix.shape)

(133401, 121942)


In [136]:
sentiment_model = LogisticRegression(max_iter=50000).fit(train_matrix, train_data['sentiment'])

In [143]:
y_pred_sentiment_model = sentiment_model.predict(test_matrix)
y_true_sentiment_model = test_data['sentiment']
print('answer: the accuracy of sentiment model is',accuracy_score(y_pred_sentiment_model, y_true_sentiment_model))

answer: the accuracy of sentiment model is 0.9322958831819136


In [147]:
# the majority of train data
num_positive_train  = (train_data['sentiment'] == +1).sum()
num_negative_train = (train_data['sentiment'] == -1).sum()
print (num_positive_train)
print (num_negative_train)

112170
21231


In [153]:
num_positive_test  = (test_data['sentiment'] == +1).sum()
num_negative_test = (test_data['sentiment'] == -1).sum()

In [154]:
accuracy_majority_class_classifier = num_positive_test/(num_negative_test+num_positive_test)
accuracy_majority_class_classifier

0.8422236214806152

## Diaper Champ Reviews
Just like we did for the reviews for the giraffe toy in the Jupyter Notebook in the lecture video, before we start our analysis you should select all reviews where the product name is ‘Baby Trend Diaper Champ’.  Let’s call this table diaper_champ_reviews.

In [158]:
diaper_champ_reviews = test_data[test_data['name']=='Baby Trend Diaper Champ']
diaper_champ_reviews['review_clean']

527    I bought this trying to save a few dollars and...
436    I love this diaper pail Yea if you let poopy d...
321    I am so glad I got the Diaper Champ instead of...
418    This has been a fantastic diaper pail for us  ...
522    The Diaper Champ is TERRIBLE at keeping the sm...
434    This is an excellent idea having the lid to di...
338    We originally had the diaper genie which we fo...
330    Although I had a Diaper Genie from my first ch...
615    I bought the Diaper Champ because it was recom...
454    After Our Diaper Genie broke after 2 years I w...
576    I just wanted to say that I love my Diaper Cha...
627    I have been using the Diaper Champ for the pas...
468    My husband and I originally registered for the...
435    It is so easy to use You never have to change ...
448    GOOD POINTSEasy to operate and changeLess expe...
534    Wow  This is fabulous  It was a tossup between...
369    We wanted to have a convenient locking recepta...
475    We have ours in our livi

Again, just as in the video, use the sentiment_model to predict the sentiment of each review in diaper_champ_reviews and sort the results according to their ‘predicted_sentiment’.

In [174]:
test_matrix_diaper_champ = vectorizer.transform(diaper_champ_reviews['review_clean'])

In [181]:
diaper_sentiment_model_pred_proba = sentiment_model.predict_proba(test_matrix_diaper_champ)
diaper_sentiment_model_pred = sentiment_model.predict(test_matrix_diaper_champ)

In [182]:
diaper_review_sentiment_pred_proba = pd.DataFrame(
    {'review_clean': diaper_champ_reviews['review_clean'],
     'predicted_sentiment_prob': diaper_sentiment_model_pred_proba[:,1].tolist(),
     'predicted_sentiment': diaper_sentiment_model_pred.tolist()
    })

In [183]:
diaper_review_sentiment_pred_proba.sort_values(by='predicted_sentiment_prob', ascending=False)

,review_clean,predicted_sentiment_prob,predicted_sentiment
513,I have never written a review for Amazon but I...,1.000000,1
559,Ive worked with kids more than half my life Fi...,0.999999,1
420,Baby Luke can turn a clean diaper to a dirty d...,0.999997,1
330,Although I had a Diaper Genie from my first ch...,0.999993,1
547,I received the Diaper Champ at my baby shower ...,0.999992,1
509,My husband and I wanted this style vs the Diap...,0.999986,1
625,We are using the Diaper Champ with our first b...,0.999913,1
418,This has been a fantastic diaper pail for us ...,0.999780,1
315,Excellent diaper disposal unit I used it in c...,0.999738,1
584,We have been using a d champ for nearly four y...,0.999322,1


What is the ‘predicted_sentiment’ for the most positive review for ‘Baby Trend Diaper Champ’ according to the sentiment_model from the Jupyter Notebook from lecture?  Save this result to answer the quiz at the end.

In [184]:
diaper_review_sentiment_pred_proba.sort_values(by='predicted_sentiment_prob', ascending=False).iloc[0]['review_clean']

'I have never written a review for Amazon but I saw some of the poor reviews on the Diaper Champ and had to add my two cents   When my sister got pregnant she could not decide if she should register for the Diaper Champ or the Diaper Genie  She left it up to me and after reviewing both items I went with the Diaper Champ     The idea of having to continually purchase bags for the Diaper Genie seemed like a waste when the Diaper Champ lets you use any type of bag     My nephew is now over two years old and while he has begun potty training my sister still uses the Diaper Champ I gave her   When it came time for me to choose a diaper disposal system I put the Diaper Champ on my registry  Yes it can be a bit tricky to open I found using two hands works best but then again do I want something that is holding wet and otherwise dirty diapers to open easily  True the mechanism sometimes wont swing the dirty diaper into the pail but HELLO that is a sign that it is probably time to put in a new 

Now use the selected_words_model you learned using just the selected_words to predict the sentiment most positive review you found above. Hint: if you sorted the diaper_champ_reviews in descending order (from most positive to most negative), this command will be helpful to make the prediction you need:

In [186]:
diaper_selected_words_model_pred_proba = selected_words_model.predict_proba(diaper_champ_reviews_test[selected_words])
diaper_selected_words_model_pred = selected_words_model.predict(diaper_champ_reviews_test[selected_words])

In [187]:
diaper_selected_words_model_pred_proba = pd.DataFrame(
    {'review_clean': diaper_champ_reviews['review_clean'],
     'predicted_sentiment_prob': diaper_selected_words_model_pred_proba[:,1].tolist(),
     'predicted_sentiment': diaper_selected_words_model_pred.tolist()
    })

In [188]:
diaper_selected_words_model_pred_proba.sort_values(by='predicted_sentiment_prob', ascending=False).iloc[0]['review_clean']

'I have a twoyearold son and I love the Diaper Champ We initially bought a Diaper Genie and used it for the first 9 months or so but we really disliked it It was difficult to use loud and then broke on us We switched the Diaper Champ and love it It contains our toddlers odors just fine We change the bag once a week and when we change a diaper we make sure to roll it up tightly and use the sticky tabs to close it up'